# Stock Market Indices - Ticker Web Scraping and Data download

*** 
## Load required libraries

In [1]:
import  pylab as pl

from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np

from scipy.cluster.vq import kmeans,vq
from math import sqrt

from sklearn.cluster import KMeans

import pandas as pd
import pandas_datareader as dr

from matplotlib import pyplot as plt

***
## Scrape ticker data
There is no consistency among wiki pages that list index constituents. You will need to adjust your code to make sure you can download most of the available data. 

### 1. Execute **one** of the four market cells below:

In [2]:
# US market - S&P500
index_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies' 
ticker_column_name='Symbol'  # This is the name of the column containing tickers on the Wikipedia page. 
yahoo_suffix=''
table_idx=0

In [14]:
# Brazilian stock exchange - Ibovespa
index_url = 'https://en.wikipedia.org/wiki/List_of_companies_listed_on_B3' 
ticker_column_name='Ticker'
yahoo_suffix='.SA'
table_idx=0

In [24]:
# UK - FTSE
index_url = 'https://en.wikipedia.org/wiki/FTSE_250_Index#List_of_FTSE_250_Index_companies'
ticker_column_name='Ticker[3]' #'Ticker'
yahoo_suffix='.L'
table_idx=1 #0

In [9]:
# Canada - TSX
index_url = 'https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index'
ticker_column_name='Ticker'
yahoo_suffix='.TO'
table_idx=1  # for example, in this link, the list of companies is located in the second table

### 2. Scrape ticker data from the link

In [25]:
# read in the url and scrape ticker data
data_table = pd.read_html(index_url,keep_default_na=False)
# Option keep_default_na=False prevents pandas from reading “NA” as NaN
# This came up when reading in TSX data for ticker "NA" for National Bank of Canada. The ticker was read as nan value

# get the list of ticker symbols
tickers = data_table[table_idx][ticker_column_name].tolist()

In [26]:
# data_table

In [27]:
tickers[0:20]

['3IN',
 'FOUR',
 '888',
 'ASL',
 'ATST',
 'ATT',
 'APAX',
 'ASCL',
 'ASHM',
 'AGR',
 'AML',
 'ATG',
 'AGT',
 'BAB',
 'BGFD',
 'BGS',
 'USA',
 'BBY',
 'BCG',
 'BNKR']

### 3. Amend tickers to be consistent with the Yahoo server

In [28]:
# Replace substring in list of strings 
# TCL.A - Transcontinental Inc. A-class shares is TCL-A.TO
adjusted_tickers = list(map(lambda st: str.replace(st, ".", "-"), tickers)) 

# For Yahoo tickers, some countries are listed with a specific suffix, e.g., Brasil's Sao Paolo has .SA
suffixed_tickers = [sub + yahoo_suffix for sub in adjusted_tickers]

suffixed_tickers[0:20]

['3IN.L',
 'FOUR.L',
 '888.L',
 'ASL.L',
 'ATST.L',
 'ATT.L',
 'APAX.L',
 'ASCL.L',
 'ASHM.L',
 'AGR.L',
 'AML.L',
 'ATG.L',
 'AGT.L',
 'BAB.L',
 'BGFD.L',
 'BGS.L',
 'USA.L',
 'BBY.L',
 'BCG.L',
 'BNKR.L']

***

## Download data based on selected tickers

In [29]:
# First, try to download a small sample of stocks - for example the first 10 on the list
# Note, some indices are large and it may take 30-45 min for daily price data to download.

prices_list = []
for ticker in suffixed_tickers[0:10]:
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)

prices_df.sort_index(inplace=True)

prices_df.head()

,3IN.L,FOUR.L,888.L,ASL.L,ATST.L,ATT.L,APAX.L,ASCL.L,ASHM.L,AGR.L
Date,,,,,,,,,,
2017-01-03,167.031357,1618.232300,184.556702,1011.108765,582.686035,83.025002,105.074165,249.920837,229.967346,45.862877
2017-01-04,166.673080,1674.033203,184.979050,1007.465088,587.252625,83.150002,105.261116,248.161530,239.871323,46.433113
2017-01-05,167.300018,1623.632324,184.556702,1010.197937,589.992615,83.000000,105.822029,252.883987,244.702621,46.066536
2017-01-06,167.210419,1642.532715,183.923218,1011.108765,592.732422,83.500000,105.822029,258.254669,240.998703,46.433113
2017-01-09,167.031357,1663.233032,187.090668,1008.376099,600.495483,84.199997,106.195946,262.051178,229.886810,46.188725


In [19]:
# If the small set download went smoothly, download prices for all tickers:

prices_list = []
for ticker in suffixed_tickers:
    try:
        prices = dr.DataReader(ticker,'yahoo','01/01/2017')['Adj Close']
        prices = pd.DataFrame(prices)
        prices.columns = [ticker]
        prices_list.append(prices)
    except:
        pass
    prices_df = pd.concat(prices_list,axis=1)

prices_df.sort_index(inplace=True)

prices_df.head()

,ABEV3.SA,AZUL4.SA,BTOW3.SA,B3SA3.SA,BBAS3.SA,BBSE3.SA,BBDC3.SA,BBDC4.SA,BRAP4.SA,BRML3.SA,...,SULA11.SA,SUZB3.SA,TAEE11.SA,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VVAR3.SA,YDUQ3.SA,WEGE3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,14.513429,NaN,9.846182,14.533460,22.744703,19.424726,15.418896,15.109305,11.927128,9.644119,...,12.809804,19.289434,14.529735,7.558028,30.379192,3.957404,20.634167,3.781286,13.660166,10.887802
2017-01-03,14.807076,NaN,10.097163,15.131575,23.785305,20.115974,16.043753,15.749231,12.428889,10.171036,...,13.431998,19.289434,14.556887,7.945701,30.871349,4.161094,21.548126,3.701680,14.420535,11.435801
2017-01-04,14.691399,NaN,10.328838,15.417248,23.661425,20.751364,16.038401,15.649483,12.157444,10.155068,...,13.929747,19.289434,14.427888,7.897643,30.379192,4.413282,21.161137,3.771335,14.349805,11.161800
2017-01-05,14.771483,NaN,10.637737,15.229776,23.603613,20.332424,16.129404,15.822728,12.766138,10.386590,...,13.900469,19.289434,14.495783,7.939295,30.558161,4.461779,21.968058,3.781286,14.119925,11.132960
2017-01-06,14.646902,NaN,10.309531,15.220850,23.859634,20.018223,16.252529,15.922472,12.544048,10.282804,...,13.790669,19.289434,14.373570,7.692592,30.423937,4.335685,21.383450,3.741483,14.013826,11.226693
